# New Template

# Setup
Specify some paths and names in the cell below.

In [1]:
configDir = "T:/Data/"  # Path to folder where you saved your wildlifeconfig file.
working_directory = "T:/temp/"
filter_set_json = ""
taxon_json = ""
EBD_file = "T:/data/eBird/ebd_US_relDec-2020.txt"
query_name = 'withRTest'
use_eBird = True
use_GBIF = False

### GBIF request method
Whether to get records from GBIF in a darwin core archive.  "False" uses the GBIF API, which has limitations that may be important.  "True" requests results be emailed in a darwin core archive.

In [2]:
get_dwca = True 

### An important, temporary bug fix
There is a bug with mpl_toolkits, the following code is a temporary fix until they resolve the bug.
https://stackoverflow.com/questions/52911232/basemap-library-using-anaconda-jupyter-notebooks-keyerror-proj-lib/54087410#54087410

os.environ['PROJ_LIB'] should be set equal to something like "c:/Users/jramone/AppData/Local/Continuum/miniconda3/envs/wrangler/Library/share"

In [3]:
#import os
#os.environ['PROJ_LIB'] = "c:/Users/nmtarr/AppData/Local/Continuum/miniconda3/envs/wrangler/Library/share"

Nothing to fill out in the next cell

In [4]:
%matplotlib inline
import sqlite3
import pprint
import json
import pandas as pd
import numpy as np
pd.set_option('display.width', 600)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 150)
from IPython.display import Image
from pygbif import occurrences
import matplotlib.pyplot as plt
import os
from datetime import datetime
t1 = datetime.now()
import sys
sys.path.append(configDir)
import wranglerconfig as config
codeDir = config.codeDir
sys.path.append(codeDir)
import wrangler_functions as functions
outDir = config.workDir
spdb = outDir + query_name + '.sqlite'
username = config.gbif_username
password = config.gbif_password
email = config.gbif_email
print("Notebook run " + str(t1))
print(spdb)

Notebook run 2021-03-25 15:09:26.895050
T:/Occurrence_Records/withRTest.sqlite


# Specify a taxon
GBIF and eBird Basic Dataset can currently be accessed.  Specificy the appropriate identifiers in the cell below.

NOTE -- cautions about taxon concept mismatches and errors

In [5]:
your_taxon_id = "TestCuckoo"
gbif_id = 2496287 
ebird_id = "Yellow-billed Cuckoo"
taxon_polygon = "POLYGON ((-84.09680233298448 36.69265225442667, -84.07962135716329 34.5561660300382, -84.07962135716329 34.5561660300382, -80.25685423694925 34.65515526072436, -81.15026497965096 36.71331438415306, -84.09680233298448 36.69265225442667))"

In [6]:
# If a json was provided, use it, otherwise create a new one with info that was provided.
if taxon_json == "":
    # Build a species dictionary
    taxon_info = {"ID": your_taxon_id, "GBIF_ID": gbif_id, "EBIRD_ID": ebird_id, 
                  "TAXON_EOO": taxon_polygon}

    # Save as json object
    out_file = open(outDir + your_taxon_id + ".json", "w")  
    json.dump(taxon_info, out_file) 
    out_file.close() 

if taxon_json != "":
    with open(taxon_json, "r") as f:
        taxon_info = json.load(f)
        f.close()
        
pprint.pprint(taxon_info)

{'EBIRD_ID': 'Yellow-billed Cuckoo',
 'GBIF_ID': 2496287,
 'ID': 'TestCuckoo',
 'TAXON_EOO': 'POLYGON ((-84.09680233298448 36.69265225442667, '
              '-84.07962135716329 34.5561660300382, -84.07962135716329 '
              '34.5561660300382, -80.25685423694925 34.65515526072436, '
              '-81.15026497965096 36.71331438415306, -84.09680233298448 '
              '36.69265225442667))'}


# Specify a filter set
Fill out this section to specify how you want records filtered and cleaned.  Alternatively, you can load a filter set here by specifying a path. Format should be ..........

In [7]:
filter_set_name = "test_filters_1"

### Date limits
Enter year and month ranges.  For example, years_range = 2015,2017 and months_range = 3,6

Justification: 

In [8]:
years_range = "2015,2020"
months_range = "9,11"

### Country
Notes:

In [9]:
country = "US"

### Bounding box
Coordinates should correspond to WGS84 (EPSG:4326).  Don't use this option if you specify a query polygon below.

Notes:

In [10]:
lat_range = "27,41"
lon_range = "-89,-75"

### Geoissue
Are GBIF records with noted geoissues OK to include? GBIF only.

Notes:

In [11]:
geoissue = False

### Collections
List collection codes that you'd like to omit. GBIF only.

Notes:

In [12]:
collection_codes_omit = ""

### Institutions
List institution codes that you'd like to omit. GBIF only

Notes:

In [13]:
institutions_omit = ""

### Datasets
List datasets that you'd like to omit.
#### ebird project here?

Notes:

In [14]:
datasets_omit = ""

### Coordinate uncertainty
Do you want to remove records without coordinate uncertainty (1) or leave them in the data set (0)?  Note that eBird records in GBIF (EOD) do not have this and neither do data in the EBD dataset.  With the EBD, the length of traveling counts is used as a surrogate value /??!!!!!?????.  

max_coordinate_uncertainty must be an integer greater than 0.

default_coordUncertainty -- coordinateUncertaintyInMeters is often not provided.  Here is an option to use a default.  If you don't want anything entered, set this equal to False (boolean, not string).

A maximum for coordinate uncertainty can also be set in meters.

Notes:

In [15]:
has_coordinate_uncertainty = False
default_coordUncertainty = 1000
max_coordinate_uncertainty = 10000

### Bases
List bases of records that you want to omit.  GBIF only.

Notes:

In [16]:
bases_omit = ""

### Sampling protocols
List sampling protocols that you would like to omit.  Options for eBird are ........._______

Notes:

In [17]:
sampling_protocols_omit = ""

### Issues
List issues that you want to omit.  GBIF only.

Notes:

In [18]:
issues_omit = ""

### Geometry (polygons)
Specify polygons to use for spatial filtering.  Records with coordinates outside of the polygons will be removed.  You can specify a geometry for the query and one for the species.  The species geometry is included to facilitate better handling of taxonomic issues.  If both are provided, the intersection is calculated and used as the filter.  The format should be well known text in WGS84 (EPSG 4326), and very importantly, vertices need to be listed counter-clockwise.  See the ccw_wkt_from_shp() function in wrangler functions for help.

Notes:

In [19]:
query_polygon = "POLYGON ((-82.74809573102132 36.96082629937069, -85.0932989306133 35.63154639485496, -81.0987220521874 33.56697226279766, -79.4235769096217 36.34054727735634, -79.4235769096217 36.34054727735634, -82.74809573102132 36.96082629937069))"

### Duplicates
Specify whether duplicates on latitude, longitude, and date should be included.

Notes:

In [20]:
duplicates_OK = False

### Save or load the filter set

In [21]:
if filter_set_json == "":
    # Build a filter set dictionary
    filter_set = {"name": filter_set_name, "query_polygon": query_polygon, "issues": issues_omit,
                  "sampling_protocols_omit": sampling_protocols_omit, "bases_omit": bases_omit,
                  "has_coordinate_uncertainty": has_coordinate_uncertainty,
                  "default_coordUncertainty": default_coordUncertainty,
                  "max_coordinate_uncertainty": max_coordinate_uncertainty,
                  "datasets_omit": datasets_omit, "collection_codes_omit": collection_codes_omit,
                  "institutions_omit": institutions_omit, "geoissue": geoissue,
                  "lat_range": lat_range, "lon_range": lon_range, "country": country, 
                  "years_range": years_range, "months_range": months_range, "duplicates_OK": duplicates_OK, "get_dwca": get_dwca}

    # Save as json object
    with open(outDir + filter_set_name + ".json", "w") as f:
        json.dump(filter_set, f) 
        f.close()
        
if filter_set_json != "":
    with open(filter_set_json, "r") as f:
        filter_set = json.load(f)
        f.close()
        
pprint.pprint(filter_set)

{'bases_omit': '',
 'collection_codes_omit': '',
 'country': 'US',
 'datasets_omit': '',
 'default_coordUncertainty': 1000,
 'duplicates_OK': False,
 'geoissue': False,
 'get_dwca': True,
 'has_coordinate_uncertainty': False,
 'institutions_omit': '',
 'issues': '',
 'lat_range': '27,41',
 'lon_range': '-89,-75',
 'max_coordinate_uncertainty': 10000,
 'months_range': '9,11',
 'name': 'test_filters_1',
 'query_polygon': 'POLYGON ((-82.74809573102132 36.96082629937069, '
                  '-85.0932989306133 35.63154639485496, -81.0987220521874 '
                  '33.56697226279766, -79.4235769096217 36.34054727735634, '
                  '-79.4235769096217 36.34054727735634, -82.74809573102132 '
                  '36.96082629937069))',
 'sampling_protocols_omit': '',
 'years_range': '2015,2020'}


## Run Processes

In [22]:
# Create an output database
functions.build_output_db(spdb)

In [25]:
# Run the appropriate queries
if use_eBird == True and use_GBIF == True:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)
    
    # Omit Cornell Lab of Ornithology records when querying GBIF
    #  collectionCode like "EBIRD*" should be omited.  HOW???>>>>>>>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    '''
    filter_set2 = filter_set
    filter_set2["collection_codes_omit"] = filter_set["collection_codes_omit"]''' 
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, working_directory, username, password, email, 
                                           dwca_download=get_dwca) 

elif use_eBird == True and use_GBIF == False:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)

elif use_eBird == False and use_GBIF == True:
    # Run GBIF query
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, working_directory, username, password, email, 
                                           dwca_download=get_dwca)
    
# Filter the returned records

# Handle duplicates

# Insert records

# Buffer points and make a shapefile

In [26]:
ebird_data.head()

,eBird_sp_code,global_unique_identifier,checklist_id,project_code,last_edited_date,common_name,observation_count,locality,latitude,longitude,observation_date,observer_id,effort_distance_km,protocol_type,effort_area_ha,trip_comments,species_comments,effort_distance_m,coordinateUncertaintyInMeters
0,yebcuc,URN:CornellLabOfOrnitholog...,S25082628,EBIRD,2018-06-02 23:01:47,Yellow-billed Cuckoo,1,Catawba River Greenway--Gr...,35.740695,-81.716223,2015-09-18,obsr119760,1.609,Traveling,NaN,85F sunny clear and calm. ...,NaN,1609.0,1629.0
1,yebcuc,URN:CornellLabOfOrnitholog...,S25029583,EBIRD,2018-08-06 21:01:39,Yellow-billed Cuckoo,1,Colonel Francis Beatty Park,35.064568,-80.742388,2015-09-14,obsr171335,3.219,Traveling,NaN,Submitted from BirdLog NA...,NaN,3219.0,3239.0
2,yebcuc,URN:CornellLabOfOrnitholog...,S25258303,EBIRD,2015-10-01 20:20:57,Yellow-billed Cuckoo,2,Jackson Park,35.321770,-82.448387,2015-10-01,obsr324369,3.862,Traveling,NaN,Birding with Wayne Forsythe,NaN,3862.0,3882.0
3,yebcuc,URN:CornellLabOfOrnitholog...,S25307631,EBIRD,2018-08-06 21:01:56,Yellow-billed Cuckoo,1,Leicester patch,35.692954,-82.694558,2015-10-05,obsr237758,1.287,Traveling,NaN,<br />Submitted from eBird...,NaN,1287.0,1307.0
4,yebcuc,URN:CornellLabOfOrnitholog...,S24914938,EBIRD,2015-09-06 16:45:10,Yellow-billed Cuckoo,1,Hunting Creek Preserve,36.033952,-80.734706,2015-09-06,obsr191211,2.414,Traveling,NaN,<br />Submitted from eBird...,NaN,2414.0,2434.0


In [ ]:
gbif_data.head()

### Connect to output database, record species and filter information.

In [ ]:
conn_occ= sqlite3.connect(spdb)
curs_occ = conn_occ.cursor()
sp_df.to_sql(name='taxon_concept', con=conn_occ, if_exists='replace')
request_df.to_sql(name=gbif_req_id, con=conn_occ, if_exists='replace')
filter_df.to_sql(name=gbif_filter_id, con=conn_occ, if_exists='replace')

### How many records made it through the filters?
This is the number of species occurrence records that were actually saved in the occurrence record sqlite database.

In [ ]:
record_count = curs_occ.execute("SELECT COUNT(occ_id) FROM occurrences WHERE taxon_id = '{0}'".format(taxon_id)).fetchone()
print(str(record_count[0]) + " records")

# Sources
#### Pre-filters

In [ ]:
tables = cursorjup.execute("SELECT table_name FROM table_descriptions").fetchall()
tables = [x[0] for x in tables]

filter_sets = [gbif_req_id, gbif_filter_id]

sources = []
for s in filter_sets:
    s = s.strip()
    for tab in tables:
        columns = cursorjup.execute("SELECT column_name FROM column_descriptions WHERE table_name = '{0}'".format(tab)).fetchall()
        columns = [x[0] for x in columns]
        for col in columns:
            try:
                a = cursorjup.execute("SELECT source FROM {1} WHERE {2} = '{0}'".format(s, tab, col)).fetchone()[0]
                sources.append(a)
            except:
                pass
print(list(set(sources))[0])

sources = pd.read_sql(sql="SELECT * FROM pre_filter_source_counts;", con=conn_occ)
print(sources)

#### Post-filters

In [ ]:
tables = cursorjup.execute("SELECT table_name FROM table_descriptions").fetchall()
tables = [x[0] for x in tables]

filter_sets = [gbif_req_id, gbif_filter_id]

sources = []
for s in filter_sets:
    s = s.strip()
    for tab in tables:
        columns = cursorjup.execute("SELECT column_name FROM column_descriptions WHERE table_name = '{0}'".format(tab)).fetchall()
        columns = [x[0] for x in columns]
        for col in columns:
            try:
                a = cursorjup.execute("SELECT source FROM {1} WHERE {2} = '{0}'".format(s, tab, col)).fetchone()[0]
                sources.append(a)
            except:
                pass
print(list(set(sources))[0])

sql = "SELECT institutionCode, collectionCode, datasetName, COUNT(occ_id) FROM occurrences GROUP BY institutionCode, collectionCode, datasetName;"
sources = pd.read_sql(sql=sql, con=conn_occ)
print(sources)

# Bases
#### Pre-filter

In [ ]:
bases = pd.read_sql(sql="SELECT value as basisOfRecord, count FROM pre_filter_value_counts WHERE attribute = 'bases';", con=conn_occ)
print(bases)

#### Post-filters

In [ ]:
sql = "SELECT basisOfRecord, COUNT(occ_id) as count FROM occurrences GROUP BY basisOfRecord;"
bases = pd.read_sql(sql=sql, con=conn_occ)
print(bases)

# Protocols
#### Pre-filters

In [ ]:
protocol = pd.read_sql(sql="SELECT value as samplingProtocol, count FROM pre_filter_value_counts WHERE attribute = 'samplingProtocols';", con=conn_occ)
print(protocol)

#### Post-filters

In [ ]:
sql = "SELECT samplingProtocol, COUNT(occ_id) as count FROM occurrences GROUP BY samplingProtocol;"
print(pd.read_sql(sql=sql, con=conn_occ))

# Issues
#### Pre-filters

In [ ]:
iss = pd.read_sql(sql="SELECT value as issues, count FROM pre_filter_value_counts WHERE attribute = 'issues';", con=conn_occ)
pd.set_option('display.max_colwidth', 80)
print(iss)

#### Post-filters

In [ ]:
sql = "SELECT issues, COUNT(occ_id) as count FROM occurrences GROUP BY issues;"
print(pd.read_sql(sql=sql, con=conn_occ))

# Descriptions of filtered records

### Locations

In [ ]:
shp1 = {'file': '{0}{1}_polygons'.format(outDir, summary_name), 'column': None,
        'alias': 'Occurrence records', 'drawbounds': True, 'linewidth': .75, 'linecolor': 'magenta',
        'fillcolor': None, 'marker':'o'}

# Display occurrence polygons
map_these=[shp1]
    
title="{1} ({0})".format(years, common_name)
try:
    functions.MapShapefilePolygons(map_these=map_these, title=title)
except Exception as e:
    print("Unable to map:  " + str(e))

### Years represented

In [ ]:
occ_years = [int(x[0]) for x in curs_occ.execute("SELECT strftime('%Y', occurrenceDate) FROM occurrences").fetchall()]
years = connjup.execute("SELECT years_range FROM gbif_requests WHERE request_id = '{0}'".format(gbif_req_id)).fetchone()[0]
years = years.split(',')
yearsrng = list(range(int(years[0]), int(years[1]), 1))
binsnum = int(years[1]) - int(years[0])
plt.hist(occ_years, bins=binsnum)
plt.ylabel("number of records")
plt.xlabel("year")
plt.xticks(yearsrng, rotation=90)
plt.title("Occurrences per Year")

### Months represented

In [ ]:
occ_months = [int(x[0]) for x in curs_occ.execute("SELECT strftime('%m', occurrenceDate) FROM occurrences").fetchall()]
plt.hist(occ_months, bins=range(1, 14), color="g")
plt.ylabel("number of records")
plt.xlabel("month")
plt.xticks(range(1, 13))
plt.title("Occurrences per Month")

### Distribution of coordinate uncertainty values

In [ ]:
occ_cert = [int(x[0]) for x in curs_occ.execute("SELECT coordinateUncertaintyInMeters FROM occurrences").fetchall()]
maxi = np.max(occ_cert)
plt.figure(figsize=(16,4))
plt.hist(occ_cert, bins=50, color="r")
plt.xticks(range(0, maxi, int(maxi/50)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("meters")
plt.title("Coordinate Uncertainty")

In [ ]:
occ_cert = [int(x[0]) for x in curs_occ.execute("SELECT coordinateUncertaintyInMeters FROM occurrences").fetchall()]
plt.figure(figsize=(16,4))
plt.boxplot(occ_cert, vert=False)
plt.xlabel("meters")
plt.title("Coordinate Uncertainty")

In [ ]:
rng_max = 2000
occ_cert2 = [x for x in occ_cert if x <= rng_max]
plt.figure(figsize=(12,4))
plt.hist(occ_cert2, bins=30, color="m", align='mid')
plt.xticks(range(0, rng_max + 100, int(rng_max/30.)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("meters")
plt.title("Coordinate Uncertainties Below 2km")

### Establishment means reported

In [ ]:
establishment = curs_occ.execute("SELECT vals FROM unique_values WHERE field = 'establishment' AND step = 'filter';").fetchall()[0]
for est in establishment:
    est = est.replace('[', '').strip().replace(']', '').replace("'", "")
    print(est)

### Identification qualifiers included

In [ ]:
quali = curs_occ.execute("SELECT DISTINCT vals FROM unique_values WHERE field = 'IDqualifier' AND step = 'filter';").fetchall()[0]
for q in quali:
    q = q.replace('[', '').strip().replace(']', '').replace("'", "")
    print(q)

### Remarks included

In [ ]:
remarks = curs_occ.execute("SELECT DISTINCT remarks FROM occurrences;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

### Attributes returned for the records in the request (pre-filters)

In [ ]:
fields_summary = pd.read_sql("SELECT * FROM gbif_fields_returned", conn_occ)#, index_col='index')
fields_summary.index.name = 'Field'
pd.set_option('display.max_rows', 250)
print(fields_summary)

### Citations for records downloaded from GBIF

In [ ]:
if get_dwca == True:
    print(curs_occ.execute("SELECT citations FROM GBIF_download_info").fetchall()[0][0])
else:
    print("Set 'get_dwca' to True to acquire a list of citations")

### Rights associated with records downloaded from GBIF

In [ ]:
if get_dwca == True:
    print(curs_occ.execute("SELECT rights FROM GBIF_download_info").fetchall()[0][0])
else:
    print("Set 'get_dwca' to True to see the rights")

### GBIF download doi

In [ ]:
if get_dwca == True:
    doi = curs_occ.execute("SELECT doi FROM GBIF_download_info").fetchall()[0][0]
    print("https://doi.org/" + doi)
else:
    print("Set 'get_dwca' to True to perform a search with a doi assigned")

### GBIF Download Key

In [ ]:
if get_dwca == True:
    print(curs_occ.execute("SELECT download_key FROM GBIF_download_info").fetchall()[0][0])
else:
    print("Set 'get_dwca' to True to perform a search with a download key assigned")

### Runtime

In [ ]:
t2 = datetime.now()
print(t2 - t1)